In [1]:
import os
os.chdir("../")

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import json
from functools import partial

In [5]:
# spark_session = SparkSession.builder.appName("Test").getOrCreate()
spark_session = (
    SparkSession.builder.appName("Python Spark SQL Hive integration example")
    .config("spark.executor.cores", 8)
    .config("spark.task.cpus", 8)
    .config("spark.cores.max", 24)
    .config("spark.driver.extraClassPath", "../jars/postgresql-42.3.3.jar")
    .getOrCreate()
)


In [6]:
analytics = spark_session.read.csv("datasets/analytics.csv", quote="\"", escape="\"", header=True)

In [7]:
transactions = spark_session.read.csv("datasets/transactions.csv", quote="\"", escape="\"", header=True)

In [8]:
users = spark_session.read.csv("datasets/users.csv", quote="\"", escape="\"", header=True)

In [9]:
users = users.to_pandas_on_spark()

In [10]:
analytics = analytics.to_pandas_on_spark()

In [11]:
transactions = transactions.to_pandas_on_spark()

In [12]:
from utils.json_utils import parse_json
from schemas.users import USERS_FIELD_DATA_SCHEMA
from schemas.transactions import TRANSACTIONS_FIELD_DATA_SCHEMA

In [13]:
def expand_fields(x, schema):
    data = json.loads(x["data"])
    data = parse_json(data, schema)
    for i in schema:
        x[i] = data[i]
    return x

In [14]:
users_data_field = partial(expand_fields, schema=USERS_FIELD_DATA_SCHEMA)
transactions_data_field = partial(expand_fields, schema=TRANSACTIONS_FIELD_DATA_SCHEMA)

In [15]:
users = users.apply(users_data_field, axis=1).drop(columns=["data"])

In [16]:
transactions = transactions.apply(transactions_data_field, axis=1).drop(columns=["data"])

In [18]:
# users.document_id.nunique()

In [20]:
# users.to_parquet("datasets/tmp.parquet")

In [23]:
# users.to_spark().write.format("parquet").save("tmp.parquet")

In [ ]:
users